### 1. 학습/테스트 데이터 세트 분리 - train_test_split()

In [1]:
# 학습 데이터 세트로만 학습/예측을 하면 예측의 의미가 없다

from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

iris = load_iris()
dt_clf = DecisionTreeClassifier()
train_data = iris.data
train_label = iris.target

## 학습데이터로 학습
dt_clf.fit(train_data, train_label)

## 학습데이터로 예측
pred = dt_clf.predict(train_data)
print('예측 정확도 : ', accuracy_score(train_label, pred))

예측 정확도 :  1.0


In [2]:
# 이를 방지하기 위해 train_test_split()메서드로 학습데이터와 테스트데이터를 분리
# train_test_split() 은
#   1) 학습용 데이터의 피처데이터 세트
#   2) 테스트용 데이터의 피처데이터 세트
#   3) 학습용 데이터의 레이블데이터 세트
#   4) 테스트용 데이터의 레이블데이터 세트

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

dt_clf = DecisionTreeClassifier()
iris_data = load_iris()

X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target,
                                                    random_state = 121, test_size = 0.3)

In [3]:
## 학습데이터로 학습
dt_clf.fit(X_train, y_train)

## 테스트 데이터로 예측
pred = dt_clf.predict(X_test)

## 예측값의 정확도 확인
print(f'예측 정확도: {accuracy_score(pred, y_test):.4f} ')

예측 정확도: 0.9556 


### 2. 교차검증

#### 2.1 K-fold 교차 검증

In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

iris = load_iris()
features = iris.data
label = iris.target
dt_clf = DecisionTreeClassifier(random_state=156)

## 5개의 fold 세트로 분리하는 객체 생성
kfold = KFold(n_splits = 5)
## 폴드 세트별 정확로를 담을 리스트 객체 생성
cv_accuracy = []

print('붓꽃 데이터 세트 크기:', features.shape[0])

붓꽃 데이터 세트 크기: 150


In [5]:
n_iter = 0

# KFold 객체의 split()메서드를 호출하면 폴드 별 학습용, 검증용 테스트의 로우 인덱스를 array로 반환
for train_index, test_index in kfold.split(features):
    ## kfold.split()으로 반환된 인덱스를 이용해 학습용, 검증용 테스트 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    ## 학습 및 예측
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    n_iter += 1
    
    ## 반복시마다 정확도 측정
    accuracy = np.round(accuracy_score(pred, y_test),4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print(f'\n#{n_iter} 교차 검증 정확도 : {accuracy}, 학습데이터 크기 : {train_size}, 검증데이터 크기 : {test_size}')
    print(f'#{n_iter} 검증 세트 인덱스 : {test_index}')
    cv_accuracy.append(accuracy)
    
# 개별 iteration별 정확도를 합하여 평균 정확도 계산
print('\n## 평균 검증 정확도:', np.mean(cv_accuracy))


#1 교차 검증 정확도 : 1.0, 학습데이터 크기 : 120, 검증데이터 크기 : 30
#1 검증 세트 인덱스 : [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]

#2 교차 검증 정확도 : 0.9667, 학습데이터 크기 : 120, 검증데이터 크기 : 30
#2 검증 세트 인덱스 : [30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53
 54 55 56 57 58 59]

#3 교차 검증 정확도 : 0.8667, 학습데이터 크기 : 120, 검증데이터 크기 : 30
#3 검증 세트 인덱스 : [60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83
 84 85 86 87 88 89]

#4 교차 검증 정확도 : 0.9333, 학습데이터 크기 : 120, 검증데이터 크기 : 30
#4 검증 세트 인덱스 : [ 90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119]

#5 교차 검증 정확도 : 0.7333, 학습데이터 크기 : 120, 검증데이터 크기 : 30
#5 검증 세트 인덱스 : [120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137
 138 139 140 141 142 143 144 145 146 147 148 149]

## 평균 검증 정확도: 0.9


#### 2.2 Stratified K fold 교차 검증
    원본데이터를 split할 때 레이블 데이터를 편향되지 않도록 골고루 분배하는 방식

In [6]:
# K-Fold의 문제점 : 레이블 값이 편향되게 분배 될 수 있음
# 아래의 결과처럼 학습레이블 데이터에 0 데이터가 없으면 0을 예측할 수 없음
import pandas as pd

iris = load_iris()
iris_df = pd.DataFrame(data = iris.data, columns = iris.feature_names)
iris_df['label'] = iris.target
iris_df['label'].value_counts()

2    50
1    50
0    50
Name: label, dtype: int64

In [7]:
kfold = KFold(n_splits=3)
n_iter = 0
for train_index, test_index in kfold.split(iris_df):
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print(f'## 교차검증: {n_iter}')
    print('학습 레이블 데이터 분포:\n', label_train.value_counts())
    print('검증 레이블 데이터 분포:\n', label_test.value_counts())

## 교차검증: 1
학습 레이블 데이터 분포:
 2    50
1    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 0    50
Name: label, dtype: int64
## 교차검증: 2
학습 레이블 데이터 분포:
 2    50
0    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 1    50
Name: label, dtype: int64
## 교차검증: 3
학습 레이블 데이터 분포:
 1    50
0    50
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    50
Name: label, dtype: int64


In [8]:
# Stratified K Fold 교차검증으로 위와 같은 문제 해결
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3)
n_iter = 0

# KFold 메서드와 형태는 비슷하지만 인자로 레이블 값을 전달해주어야 함
for train_index, test_index in skf.split(iris_df, iris_df['label']):
    n_iter += 1
    label_train = iris_df['label'].iloc[train_index]
    label_test = iris_df['label'].iloc[test_index]
    print(f'## 교차검증: {n_iter}')
    print('학습 레이블 데이터 분포:\n', label_train.value_counts())
    print('검증 레이블 데이터 분포:\n', label_test.value_counts())

## 교차검증: 1
학습 레이블 데이터 분포:
 2    34
1    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 1    17
0    17
2    16
Name: label, dtype: int64
## 교차검증: 2
학습 레이블 데이터 분포:
 1    34
2    33
0    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    17
0    17
1    16
Name: label, dtype: int64
## 교차검증: 3
학습 레이블 데이터 분포:
 0    34
2    33
1    33
Name: label, dtype: int64
검증 레이블 데이터 분포:
 2    17
1    17
0    16
Name: label, dtype: int64


In [9]:
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
# StratifiedKFold를 이용하여 붓꽃 데이터를 교차검증

dt_clf = DecisionTreeClassifier(random_state=156)

skfold = StratifiedKFold(n_splits = 3)
n_iter = 0
cv_accuracy=[]

iris_target = iris_df.drop('label',axis=1)

# StratifiedKFold의 split()호출시 반드시 레이블 데이터 세트도 추가 입력 필요
for train_index, test_index in skfold.split(iris_target, iris_df['label']):
    # split()으로 반환된 index를 이용해 학습용, 검증용 테스트 데이터 추출
    X_train, X_test = iris_target.iloc[train_index], iris_target.iloc[test_index]
    y_train, y_test = iris_df['label'].iloc[train_index], iris_df['label'].iloc[test_index]
    # 학습 및 예측
    dt_clf.fit(X_train, y_train)
    pred = dt_clf.predict(X_test)
    
    # 반복시마다 정확도 측정
    n_iter += 1
    accuracy = np.round(accuracy_score(pred, y_test),4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print(f'\n#{n_iter} 교차 검증 정확도: {accuracy}, 학습데이터 크기: {train_size}, 검증 데이터 크기: {test_size}')
    print(f'#{n_iter} 검증 세트 인덱스: {test_index}')
    cv_accuracy.append(accuracy)
    
# 교차 검증별 정확도 및 평균 정확도 계산
print('\n## 교차 검증별 정확도: ', np.round(cv_accuracy,4))
print('## 평균 검증 정확도: ', np.mean(cv_accuracy))


#1 교차 검증 정확도: 0.98, 학습데이터 크기: 100, 검증 데이터 크기: 50
#1 검증 세트 인덱스: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  50
  51  52  53  54  55  56  57  58  59  60  61  62  63  64  65  66 100 101
 102 103 104 105 106 107 108 109 110 111 112 113 114 115]

#2 교차 검증 정확도: 0.94, 학습데이터 크기: 100, 검증 데이터 크기: 50
#2 검증 세트 인덱스: [ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  67
  68  69  70  71  72  73  74  75  76  77  78  79  80  81  82 116 117 118
 119 120 121 122 123 124 125 126 127 128 129 130 131 132]

#3 교차 검증 정확도: 0.98, 학습데이터 크기: 100, 검증 데이터 크기: 50
#3 검증 세트 인덱스: [ 34  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  83  84
  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 133 134 135
 136 137 138 139 140 141 142 143 144 145 146 147 148 149]

## 교차 검증별 정확도:  [0.98 0.94 0.98]
## 평균 검증 정확도:  0.9666666666666667


In [10]:
## 모든 분류는 Stratified K Fold 교차검증을 하는 것이 좋음
## 회귀는 Stratified K Fold 지원 X

#### 2.3 cross_val_score() : 교차 검증을 간편하게 해주는 API

    cross_val_score(estimator, X, y, scoring=None, cv=None, n_jobs=1, verbose=0, fit_params=None, pre_dispatch = '2*n_jobs)
    
      주요 옵션 - 1) estimator : 지도학습 알고리즘 클래스 (분류 / 회귀)
                  2) X, y : X는 피처 데이터세트, y는 레이블 데이터세트
                  3) scoring : 예측 성능 평가 지표
                  4) cv : 교차 검증 폴드 수
      
      cross_val_score() 메서드는 내부적으로 Stratified K Fold 교차검증 방식 활용

In [11]:
# Stratified K Fold의 수행과정
#   1) 폴드 세트를 설정하여 객체생성
#   2) for 루프에서 반복으로 학습 및 테스트 데이터의 인덱스를 추출
#   3) 반복적으로 학습과 예측을 수행하고 예측성능을 반환

# 해당과정을 한번에 수행하는 cross_val_score()

In [12]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.datasets import load_iris

iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state=156)

data = iris_data.data
label = iris_data.target

scores = cross_val_score(dt_clf, data, label, scoring='accuracy', cv=3)
print('교차 검증별 정확도:', np.round(scores,4))
print('평균 검증 정확도:', np.round(scores.mean(),4))

교차 검증별 정확도: [0.98 0.94 0.98]
평균 검증 정확도: 0.9667


### 3. 교차검증과 최적 하이퍼파라미터 튜닝을 한번에 수행하는 GridSearchCV

    GridSearchCV(estimator, param_grid, scoring=None, n_jobs=None, iid='deprecated', refit=True, cv=None, 
                 verbose=0, pre_dispatch='2*n_jobs', error_score=nan, return_train_score=False)
                 
       주요 옵션 - 1) estimator : 지도학습 알고리즘 클래스(분류, 회귀, pipeline)
                   2) param_grid : 파라미터가 담겨있는 딕셔너리. 딕셔너리의 key는 파라미터명, value는 파라미터 값
                   3) scoring : 예측 성능 평가방법. 몇가지 정해진 지표가 있으나 필요시 별도의 사용자 정의 함수도 지정가능
                   4) cv : 교차 검증을 위해 불할되는 학습/테스트 세트의 개수
                   5) refit : 최적의 하이퍼파라미터를 찾은 뒤 입력된 estimator 객체를 해당 파라미터로 재학습. 디폴트가 true
                   
        주요 메서드 = 1) GridSearchCV.cv_results_ : 결과 반환, dfm으로 변환하여 해석
                      2) GridSearchCV.best_params_ : 최적파라미터 반환
                      3) GridSearchCV.best_score_ : 최고 정확도 반환
                      4) GridSearchCV.best_estimator_ : refit=True 이면 최적의 하이퍼파라미터로 설정된 estimator객체 생성

In [13]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

## 데이터를 로딩하고 학습 데이터와 테스트 데이터 분리
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state = 121)
dtree = DecisionTreeClassifier()

## 파라미터를 딕셔너리 형태로 설정
grid_parameters = {'max_depth':[1,2,3],
                   'min_samples_split':[2,3]}

In [14]:
import pandas as pd

## param_grid의 하이퍼 파라미터를 3개의 train, test set fold로 나누어 테스트수행 설정
grid_dtree = GridSearchCV(dtree, param_grid = grid_parameters, scoring = 'accuracy', cv=3, refit=True)

## 붓꽃 학습데이터로 param_grid의 하이퍼 파라미터를 순차적으로 학습/평가
grid_dtree.fit(X_train, y_train)

## GridSearchCV 결과를 추출해 DataFrame으로 반환
scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score','split0_test_score', 'split1_test_score', 'split2_test_score']]

,params,mean_test_score,rank_test_score,split0_test_score,split1_test_score,split2_test_score
0,"{'max_depth': 1, 'min_samples_split': 2}",0.700000,5,0.700,0.7,0.70
1,"{'max_depth': 1, 'min_samples_split': 3}",0.700000,5,0.700,0.7,0.70
2,"{'max_depth': 2, 'min_samples_split': 2}",0.958333,3,0.925,1.0,0.95
3,"{'max_depth': 2, 'min_samples_split': 3}",0.958333,3,0.925,1.0,0.95
4,"{'max_depth': 3, 'min_samples_split': 2}",0.975000,1,0.975,1.0,0.95
5,"{'max_depth': 3, 'min_samples_split': 3}",0.975000,1,0.975,1.0,0.95


In [15]:
print('GridSearchCV 최적 파라미터: ', grid_dtree.best_params_)
print('GridSearchCV 최고 정확도: ', grid_dtree.best_score_)

GridSearchCV 최적 파라미터:  {'max_depth': 3, 'min_samples_split': 2}
GridSearchCV 최고 정확도:  0.975


In [16]:
# 최적의 파라미터로 테스트 데이터 세트 예측

estimator = grid_dtree.best_estimator_
pred = estimator.predict(X_test)

print('테스트 데이터 세트 정확도 :',np.round(accuracy_score(pred, y_test),4))

테스트 데이터 세트 정확도 : 0.9667


### 4. model_selection의 흐름


    1) train_test_split()으로 데이터를 학습용데이터와 테스트용 데이터로 구분
    2) train 데이터에 대해 GridSearchCV 수행
        ㄱ) 최적 하이퍼 파라미터 찾기
        ㄴ) 교차 검증하기
    3) 최적 하이퍼파라미터로 모델을 생성하고 test 데이터로 예측수행
    4) accuracy_score(를 비롯한 평가지표)로 예측성능 평가 